# Adding social and economic data

In [89]:
import pandas as pd


In [75]:
emp = pd.read_excel('soc-econ/emp.xlsx')
selected_indicators = ['Уровень безработицы: Уровень безработицы']

emp = emp[emp["indicator_name"].isin(selected_indicators)]
emp = emp[emp["year"] == 2021]
emp = emp[["object_name", "indicator_value", "indicator_name"]]
emp = emp.pivot(index = 'object_name', columns= 'indicator_name', values = 'indicator_value')
emp.columns = ["unemp"]

In [76]:
inc = pd.read_excel('soc-econ/inc.xlsx')
selected_indicators = ['Численность населения с денежными доходами ниже границы бедности/величины прожиточного минимума',
                       'Медианный среднедушевой денежный доход населения']
inc = inc[inc["indicator_name"].isin(selected_indicators)]
inc = inc[inc["year"] == 2021]
inc = inc[["object_name", "indicator_value", "indicator_name"]]
inc = inc.pivot(index = 'object_name', columns= 'indicator_name', values = 'indicator_value')
inc.columns = ['median_income', "share_poverty"]

In [90]:
nat = pd.read_excel("soc-econ/ethnic_composition.xlsx", sheet_name=None)
russian_percentages = {}

for region, data in nat.items():
    total_row = data[data.iloc[:, 0].str.contains("Указавшие национальную принадлежность", na=False)]
    total_population = total_row.iloc[0, 1] if not total_row.empty else None
    
    russian_row = data[data.iloc[:, 0].str.startswith("Русские", na=False)]
    russian_population = russian_row.iloc[0, 1] if not russian_row.empty else None
    
    if total_population and russian_population:
        percentage = (russian_population / total_population)*100
        russian_percentages[region] = round(percentage, 2)
    else:
        russian_percentages[region] = "NA"

df_rus = pd.DataFrame.from_dict(russian_percentages, orient='index', columns=['% Russians'])
df_rus.index.name = 'Region'
df_rus.reset_index(inplace=True)

region_name_mapping = {
    'Архангельская область без автономного округа': 'Архангельская область без АО',
    'Ненецкий автономный округ': 'Ненецкий АО',
    'Кемеровская область - Кузбасс': 'Кемеровская область',
    'г. Москва' : 'Москва',
    'г. Санкт-Петербург': 'Санкт-Петербург',
    'г. Севастополь': 'Севастополь',
    'Еврейская автономная область': 'Еврейская АО',
    'Кабардино-Балкарская Республика': 'Кабардино-Балкария',
    'Карачаево-Черкесская Республика': 'Карачаево-Черкесия',
    'Республика Адыгея (Адыгея)': 'Республика Адыгея',
    'Республика Саха (Якутия)': 'Якутия',
    'РСО-Алания': 'Северная Осетия',
    'Республика Татарстан (Татарстан)': 'Республика Татарстан',
    'Чувашская Республика - Чувашия': 'Чувашская Республика',
    'Тюменская область без автономных округов': 'Тюменская область без АО',
    'ХМАО' : 'Ханты-Мансийский АО',
    'ЯНАО' : 'Ямало-Hенецкий АО',
    'Чукотский автономный округ': 'Чукотский АО',
}

df_rus["Region"] = df_rus["Region"].replace(region_name_mapping)

In [78]:
df_se = emp.merge(inc, on = 'object_name', how = 'left')
df_se.index.name = "Region"
df_se = df_se.reset_index()

region_name_mapping = {
    'Архангельская область без автономного округа': 'Архангельская область без АО',
    'Ненецкий автономный округ': 'Ненецкий АО',
    'Еврейская автономная область':'Еврейская АО',
    'Кемеровская область - Кузбасс': 'Кемеровская область',
    'Город Москва столица Российской Федерации город федерального значения': 'Москва',
    'Город Санкт-Петербург город федерального значения': 'Санкт-Петербург',
    'Город федерального значения Севастополь': 'Севастополь',
    'Еврейская автономная область': 'Еврейская АО',
    'Кабардино-Балкарская Республика': 'Кабардино-Балкария',
    'Карачаево-Черкесская Республика': 'Карачаево-Черкесия',
    'Республика Адыгея (Адыгея)': 'Республика Адыгея',
    'Республика Саха (Якутия)': 'Якутия',
    'Республика Северная Осетия — Алания': 'Северная Осетия',
    'Республика Татарстан (Татарстан)': 'Республика Татарстан',
    'Чувашская Республика - Чувашия': 'Чувашская Республика',
    'Тюменская область без автономных округов': 'Тюменская область без АО',
    'Ханты-Мансийский автономный округ — Югра': 'Ханты-Мансийский АО',
    'Ямало-Ненецкий автономный округ': 'Ямало-Hенецкий АО',
    'Чукотский автономный округ': 'Чукотский АО',
}

df_se["Region"] = df_se["Region"].replace(region_name_mapping)

In [92]:
df_full = df_rus.merge(df_se, on = 'Region', how = 'left')
df_full.to_csv("intermediate/soc_econ_data.csv")